In [41]:
#inititalize the notebook
import os
import textwrap
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

chat_deployment_name = "gpt-4"

print("*** init done! ***")


*** init done! ***


simple prompt

In [42]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Een HBO-ICT opleiding in Nederland duurt normaal gesproken vier jaar. Dit geldt
voor zowel voltijdse als deeltijdse studies, hoewel deeltijdse studies
flexibeler kunnen zijn qua duur. In Heerlen wordt de HBO-ICT opleiding
aangeboden door Zuyd Hogeschool, een van de hogescholen in Limburg.  Als je de
opleiding versneld wilt volgen of als je relevante certificaten of werkervaring
hebt, kun je soms vrijstellingen krijgen voor bepaalde vakken, waardoor je de
studie sneller kunt afronden.  Let wel op dat de situatie kan veranderen en het
beste is om de meest up-to-date informatie rechtstreeks te verkrijgen van Zuyd
Hogeschool of de specifieke faculteit die de opleiding aanbiedt.


system prompt

In [43]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent die antwoord als een grappige kat"
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Nou miauw, luister goed! Normaal gesproken duurt een HBO-ICT opleiding in
Nederland, en dus ook in Heerlen, zo'n vier jaar als je 't volgt in voltijd. Dat
is als alles volgens plan spint en je alle muizen, eh... ik bedoel examens,
vangt zonder problemen. Maar als je besluit om naast je kattenvrienden wat meer
tijd door te brengen achter de boeken met een deeltijdopleiding, dan kan het
langer duren, afhankelijk van hoe snel je door het materiaal krabt.  En als je
zo behendig bent als een kat en gebruik maakt van een zogenaamde 'fast track',
kan het ook zijn dat je opleiding korter duurt. Maar onthoud goed, kleine kat:
ongeacht de tijd, zorg ervoor dat je lekker in je vachtje zit tijdens de jacht
naar ICT-kennis! Miauw! 🐱💻


RESET THE SYSTEM PROMPT

In [44]:
firstQuestion = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

firstResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": firstQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(firstResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

In Nederland duurt een HBO-ICT opleiding, zoals die in Heerlen en op andere
Hogescholen, doorgaans vier jaar voor de voltijd variant. Dit is de
standaardduur voor een bacheloropleiding aan een hogeschool. Echter, er zijn ook
andere studiemogelijkheden, zoals deeltijd of duaal, waarbij de opleidingsduur
kan variëren afhankelijk van hoe de studie is georganiseerd en de omstandigheden
van de student.  Daarnaast bieden sommige hogescholen versnelde trajecten aan
voor studenten die al relevante vooropleidingen hebben afgerond of de nodige
werkervaring hebben. Dit kan er ook voor zorgen dat de opleidingsduur korter is
dan vier jaar.  Zuyd Hogeschool is een van de hogescholen in Heerlen die een
HBO-ICT opleiding aanbiedt. Het is verstandig om de specifieke details van de
opleiding te controleren op de website van de betreffende instelling of
rechtstreeks contact op te nemen met de instelling voor de meest actuele
informatie.


vervolgvraag

In [45]:
secondQuestion = "Waar is de opleiding gevestigd?"

secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Omdat je vraag vrij algemeen is en "de opleiding" naar talloze mogelijkheden kan
verwijzen, heb ik wat meer informatie nodig om je correct te kunnen helpen. Kun
je aangeven welke opleiding je precies bedoelt? Bijvoorbeeld de naam van de
instelling, de stad, het land, of het type opleiding kan helpen om een
nauwkeurig antwoord te geven.


short term memory - langchain

In [46]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()


# LLM
llm = AzureChatOpenAI(
    deployment_name=chat_deployment_name,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
) 


# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Je bent een behulpzame assistent"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
print("---------")
print(firstQuestion)
print("---------")
response = conversation({"question": firstQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)


print("---------")
print(secondQuestion + " (nu hebben we het antwoord van de eerste vraag als context)")
print("---------")
response = conversation({"question": secondQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)




---------
Hoe lang duurt een HBO-ICT opleiding in Heerlen?
---------
Een HBO-ICT opleiding in Nederland, inclusief die in Heerlen, duurt doorgaans 4
jaar als het een voltijdseopleiding betreft. Dit komt overeen met een studielast
van 240 ECTS (European Credit Transfer and Accumulation System), waarbij 60 ECTS
per studiejaar wordt gehanteerd.  In Heerlen wordt de HBO-ICT opleiding
aangeboden door Zuyd Hogeschool. Het is echter mogelijk dat er verschillende
trajecten of studievormen beschikbaar zijn, zoals deeltijd, duaal of versnelde
programma's, die de duur van de opleiding kunnen beïnvloeden. Voor de specifieke
details van de opleiding bij Zuyd Hogeschool in Heerlen, is het aan te raden om
de website van de hogeschool te raadplegen of contact op te nemen met hun
studieadviseurs.
---------
Waar is de opleiding gevestigd? (nu hebben we het antwoord van de eerste vraag als context)
---------
De HBO-ICT opleiding in Heerlen wordt aangeboden door Zuyd Hogeschool. Het adres
van de locatie w